In [6]:
tf.__version__

'2.9.2'

In [1]:
! pip install tensorflow-model-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 238 kB 7.0 MB/s 


In [3]:
import tensorflow as tf
import numpy as np


num_filters=512
inputs=tf.keras.Input((32,32,1024))

outputs= tf.keras.layers.Conv2DTranspose(filters=num_filters,
                    kernel_size=(2, 2),
                    strides=2,
                    padding="same",name='convTrans')(inputs)

base_model=tf.keras.Model(inputs=inputs,outputs=outputs,name="test_transConv2D_model")
print(base_model.layers[1].weights[0].shape)# (2, 2, 512, 1024)
print(base_model.layers[1].weights[1].shape)# (512,)`

import tensorflow_model_optimization as tfmot
quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)

x_train = np.random.randn(4,32, 32, 1024).astype(np.float32)
y_train = np.random.randn(4, 64, 64, 512).astype(np.float32)
quant_aware_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)
quant_aware_model.fit(x_train, y_train,epochs=1)
quant_aware_model.summary()

quant_aware_model.input.set_shape((1,) + quant_aware_model.input.shape[1:])

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_input_type = tf.uint8 # or tf.uint8
converter.inference_output_type = tf.uint8 # or tf.uint8
quantized_tflite_model = converter.convert()
tflite_model_filename='test_transConv2D_model.tflite'
with open(tflite_model_filename, 'wb') as f:
    f.write(quantized_tflite_model)
    print("wirte tflite file done!")

(2, 2, 512, 1024)
(512,)
1/1 [==============================] - 3s 3s/step - loss: nan - accuracy: 0.0026
Model: "test_transConv2D_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 1024)]    0         
                                                                 
 quantize_layer_1 (QuantizeL  (None, 32, 32, 1024)     3         
 ayer)                                                           
                                                                 
 quant_convTrans (QuantizeWr  (None, 64, 64, 512)      2097669   
 apperV2)                                                        
                                                                 
Total params: 2,097,672
Trainable params: 2,097,664
Non-trainable params: 8
_________________________________________________________________


wirte tflite file done!


/usr/local/lib/python3.8/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [4]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  44508      0 --:--:-- --:--:-- --:--:-- 43741
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,722 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://packages.cloud.google.com/apt coral-edgetpu-stable/main amd64 Package

In [5]:
! edgetpu_compiler -s /content/test_transConv2D_model.tflite

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 81 ms.

Input model: /content/test_transConv2D_model.tflite
Input size: 2.00MiB
Output model: test_transConv2D_model_edgetpu.tflite
Output size: 2.03MiB
On-chip memory used for caching model parameters: 0.00B
On-chip memory remaining for caching model parameters: 7.12MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 4
Operation log: test_transConv2D_model_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 1
Number of operations that will run on CPU: 3

Operator                       Count  